# 要約 
このJupyterノートブックでは、Gemma-2 9bモデルのトレーニング方法とその結果について説明されています。このノートブックの主な目的は、大規模言語モデル（LLM）のファインチューニングにおける最新の手法であるQLoRA (Quantized Low-Rank Adaptation)を使用して、競技会でのパフォーマンスを向上させることです。

### 問題点
ノートブックは、ユーザーが好みのチャットボット応答を予測するために、Gemma-2のトレーニングによる評価パフォーマンスを改善しようとしています。具体的には、評価セットのロスを最小化することで、実際の競技会や評価基準のスコア（LB: 0.941）を向上させることを目指しています。

### 使用手法とライブラリ
以下の手法およびライブラリが使用されています。

1. **QLoRA**: 低ランクの適応法を用いてモデルのトレーニングを効率化し、大規模モデルのメモリ使用を減らします。モデルの重みを量子化しつつトレーニングを行うため、計算コストを削減します。
2. **LoRA**: 通常のファインチューニングの代わりに、特定の層に小さな行列を介してパラメータを調整することで、トレーニング中に元の重みの更新を最小限に抑えます。
3. **Hugging Face Transformers**: 一般的なトランスフォーマーモデルのフレームワークとして使用され、`Gemma2ForSequenceClassification`などの特定のモデルやトークナイザーが利用されています。
4. **PyTorch**: モデルのトレーニングと評価に使用される深層学習ライブラリです。
5. **Sklearn**: 精度や対数損失の計算を行うために用いられています。

### ノートブックのフロー
- **データ準備**: Kaggleのデータセットを読み込み、カスタムトークナイザーを設定してデータを前処理します。
- **モデル設定**: Gemma-2の設定やトレーニングメソッドを指定し、LoRAおよびQLoRAのパラメータを設定します。
- **トレーニング**: 計算されたいくつかの評価指標（例えば、ロスと精度）を用いて、モデルのトレーニングを実施します。
- **結果の評価**: 最後に、評価セットに対するモデルのロスと精度を報告します。

このノートブックは、全体のプロセスを通じて、LLMのファインチューニングを改善し、実用的なパフォーマンスを提供するための具体的なステップと結果を示しています。

---


# 用語概説 
以下は、ノートブックに含まれる専門用語に関する簡単な解説です。初心者がつまずきそうな、実務経験が少ないと馴染みのない用語に焦点を当てています。

1. **LoRA (Low-Rank Adaptation)**:
   LoRAは、モデルの重みを効率的に更新する技術で、大規模なモデルに対してファインチューニングを行う際、重みの更新を行うのではなく、低ランクの行列を利用して重みの変化を表現します。これにより、メモリ使用量が削減され、トレーニングの効率が向上します。

2. **QLoRA (Quantized Low-Rank Adaptation)**:
   LoRAの拡張で、量子化技術を用いてモデルの重みをさらに低精度で圧縮します。これにより、大規模モデルの必要なメモリ量を大幅に減少させながら、通常のファインチューニングに匹敵する性能を保持します。

3. **モデルの凍結 (Freezing a model)**:
   特定のパラメータや層をトレーニング中に固定し、学習を行わないようにすることです。これにより、トレーニング時のメモリ使用量を削減し、新しく追加されたパラメータにのみ学習を集中させることができます。

4. **データコレーター (Data Collator)**:
   バッチ処理の際にデータを整形する役割を持つ機能です。複数の入力をまとめて一つのバッチとして扱うために必要な前処理を行います。特にパディング（長さを揃える作業）などを自動で行ないます。

5. **バッチサイズ (Batch Size)**:
   モデルが一度に処理するデータの数を指します。大きいバッチサイズを設定することで、トレーニングのスピードを向上させることができますが、メモリ負荷が増す可能性もあります。一方、小さいバッチサイズはメモリの負担を軽減しますが、トレーニングが遅くなることがあります。

6. **勾配蓄積 (Gradient Accumulation)**:
   小さいバッチサイズで複数回のパスを行い、その勾配を累積して、最後に一度にパラメータを更新する手法です。これにより、実質的に大きなバッチサイズを使用したかのような効果を得ることができます。

7. **ドロップアウト (Dropout)**:
   ニューラルネットワークのトレーニング時に一部のニューロンをランダムに無効化する手法です。これは過学習を防ぐために有効で、モデルの一般化性能を向上させる効果があります。

8. **トークナイザー (Tokenizer)**:
   テキストデータをモデルが処理しやすい形式に変換するためのツールです。テキストを単語やサブワードに分割し、数値として表現します。

9. **フォワード/バックワード計算**:
   フォワード計算は、モデルに入力を与えた時の出力を計算するプロセスです。バックワード計算は誤差逆伝播法を利用して、出力の誤りに基づいてモデルの重みを更新するプロセスです。

10. **ファインチューニング (Fine-tuning)**:
   すでにトレーニングされたモデルを特定のタスクに合わせて追加のトレーニングを行うプロセスです。これにより、モデルは新しいデータに対してより適した性能を発揮できます。

これらの用語は、初学者が理解を深める上で重要なものですので、しっかりと理解しておくと良いでしょう。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## What this notebook is
This notebook demonstrates how I trained Gemma-2 9b to obtain LB: 0.941. The inference code can be found [here](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora).
I used 4-bit quantized [Gemma 2 9b Instruct](https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit) uploaded by unsloth team as a base-model and added LoRA adapters and trained for 1 epoch.

## Result

I used `id % 5 == 0` as an evaluation set and used all the rest for training.

| subset | log loss |
| - | - |
| eval | 0.9371|
| LB | 0.941 |

## What is QLoRA fine-tuning?

In the conventional fine-tuning, weight ($\mathbf{W}$) is updated as follows:

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

where $L$ is a loss at this step and $\eta$ is a learning rate.

[LoRA](https://arxiv.org/abs/2106.09685) tries to approximate the $\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$ by factorizing $\Delta \mathbf{W}$ into two (much) smaller matrices, $\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$ and $\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$ with $r \ll \text{min}(\text{d}, \text{k})$.

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

During training, only $\mathbf{A}$ and $\mathbf{B}$ are updated while freezing the original weights, meaning that only a fraction (e.g. <1%) of the original weights need to be updated during training. This way, we can reduce the GPU memory usage significantly during training while achieving equivalent performance to the usual (full) fine-tuning.

[QLoRA](https://arxiv.org/abs/2305.14314) pushes the efficiency further by quantizing LLM. For example, a 8B parameter model alone would take up 32GB of VRAM in 32-bit, whereas quantized 8-bit/4-bit 8B model only need 8GB/4GB respectively. 
Note that QLoRA only quantize LLM's weights in low precision (e.g. 8-bit) while the computation of forward/backward are done in higher precision (e.g. 16-bit) and LoRA adapter's weights are also kept in higher precision.

1 epoch using A6000 took ~15h in 4-bit while 8-bit took ~24h and the difference in log loss was not significant.

## Note
It takes prohivitively long time to run full training on kaggle kernel. I recommend to use external compute resource to run the full training.
This notebook uses only 100 samples for demo purpose, but everything else is same as my setup.

</div>
<div class="column-right">

# 日本語訳

## このノートブックについて
このノートブックでは、Gemma-2 9bをどのようにトレーニングしてLB: 0.941を取得したかを示します。推論コードは[こちら](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora)にあります。
私は、unslothチームがアップロードした4ビット量子化された[Gemma 2 9b Instruct](https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit)をベースモデルとして使用し、LoRAアダプターを追加して1エポックのトレーニングを行いました。

## 結果

評価セットとして`id % 5 == 0`を使用し、残りをすべてトレーニングに使用しました。

| サブセット | ロス |
| - | - |
| eval | 0.9371 |
| LB | 0.941 |

## QLoRAファインチューニングとは？

従来のファインチューニングでは、重み（$\mathbf{W}$）は次のように更新されます：

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

ここで、$L$はこのステップでの損失、$\eta$は学習率です。

[LoRA](https://arxiv.org/abs/2106.09685)は、$\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$を2つの（はるかに）小さな行列、$\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$と$\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$に因子分解して近似しようとします。ここで$r \ll \text{min}(\text{d}, \text{k})$です。

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

トレーニング中、元の重みを凍結し、$\mathbf{A}$と$\mathbf{B}$のみが更新されるため、トレーニング中に更新する必要がある元の重みの割合はごくわずか（例：<1%）です。この方式により、トレーニング中のGPUメモリ使用量を大幅に削減しながら、通常の（フル）ファインチューニングと同じ性能を達成できます。

[QLoRA](https://arxiv.org/abs/2305.14314)は、LLMを量子化することでさらに効率を高めています。例えば、8Bパラメータモデルは32ビットで32GBのVRAMを占有しますが、量子化された8ビット/4ビットの8Bモデルはそれぞれ8GB/4GBしか必要ありません。
QLoRAは、LLMの重みを低精度（例：8ビット）で量子化する一方で、フォワード/バックワードの計算は高精度（例：16ビット）で行い、LoRAアダプターの重みも高精度で維持されることに注意してください。

4ビットでのA6000使用の1エポックは約15時間かかり、8ビットでは約24時間かかりましたが、ロスの違いは目立ちませんでした。

## 注意
Kaggleカーネルでの完全なトレーニングの実行には非常に長い時間がかかります。完全なトレーニングを実行するためには外部の計算リソースを使用することをお勧めします。
このノートブックではデモ目的でわずか100サンプルを使用していますが、その他はすべて私の設定と同じです。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# gemma-2 is available from transformers>=4.42.3
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft
```

</div>
<div class="column-right">

# 日本語訳

```python
# gemma-2はtransformers>=4.42.3から使用可能です
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft
```

</div>
</details>

In [ ]:
# gemma-2はtransformers>=4.42.3から使用可能です
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score
```

</div>
<div class="column-right">

# 日本語訳

```python
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score
```

</div>
</details>

In [ ]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Configurations

</div>
<div class="column-right">

# 日本語訳

### 設定


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 1024
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 2  # global batch size is 8 
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 16  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"
    
config = Config()
```

</div>
<div class="column-right">

# 日本語訳

```python
@dataclass
class Config:
    output_dir: str = "output"  # 出力フォルダ
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4ビット量子化されたgemma-2-9b-instruct
    max_length: int = 1024  # 最大長
    n_splits: int = 5  # データ分割数
    fold_idx: int = 0  # 現在のフォールドインデックス
    optim_type: str = "adamw_8bit"  # 使用するオプティマイザの種類
    per_device_train_batch_size: int = 2  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps: int = 2  # グローバルバッチサイズは8
    per_device_eval_batch_size: int = 8  # デバイスごとの評価バッチサイズ
    n_epochs: int = 1  # エポック数
    freeze_layers: int = 16  # 総レイヤ数は42、最初の16レイヤにはアダプターを追加しない
    lr: float = 2e-4  # 学習率
    warmup_steps: int = 20  # ウォームアップステップ数
    lora_r: int = 16  # LoRAのランク
    lora_alpha: float = lora_r * 2  # LoRAのアルファ
    lora_dropout: float = 0.05  # LoRAのドロップアウト率
    lora_bias: str = "none"  # LoRAのバイアス

config = Config()
```

</div>
</details>

In [ ]:
@dataclass
class Config:
    output_dir: str = "output"  # 出力フォルダ
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4ビット量子化されたgemma-2-9b-instruct
    max_length: int = 1024  # 最大長
    n_splits: int = 5  # データ分割数
    fold_idx: int = 0  # 現在のフォールドインデックス
    optim_type: str = "adamw_8bit"  # 使用するオプティマイザの種類
    per_device_train_batch_size: int = 2  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps: int = 2  # グローバルバッチサイズは8
    per_device_eval_batch_size: int = 8  # デバイスごとの評価バッチサイズ
    n_epochs: int = 1  # エポック数
    freeze_layers: int = 16  # 総レイヤ数は42、最初の16レイヤにはアダプターを追加しない
    lr: float = 2e-4  # 学習率
    warmup_steps: int = 20  # ウォームアップステップ数
    lora_r: int = 16  # LoRAのランク
    lora_alpha: float = lora_r * 2  # LoRAのアルファ
    lora_dropout: float = 0.05  # LoRAのドロップアウト率
    lora_bias: str = "none"  # LoRAのバイアス

config = Config()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

#### Training Arguments

</div>
<div class="column-right">

# 日本語訳

#### トレーニング引数


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=200,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
training_args = TrainingArguments(
    output_dir="output",  # 出力ディレクトリ
    overwrite_output_dir=True,  # 出力ディレクトリを上書きする
    report_to="none",  # ログの報告先
    num_train_epochs=config.n_epochs,  # トレーニングエポック数
    per_device_train_batch_size=config.per_device_train_batch_size,  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps=config.gradient_accumulation_steps,  # グラデーション蓄積ステップ数
    per_device_eval_batch_size=config.per_device_eval_batch_size,  # デバイスごとの評価バッチサイズ
    logging_steps=10,  # ログの出力間隔
    eval_strategy="epoch",  # 評価戦略
    save_strategy="steps",  # 保存戦略
    save_steps=200,  # 保存ステップ数
    optim=config.optim_type,  # オプティマイザの種類
    fp16=True,  # FP16を使用するか
    learning_rate=config.lr,  # 学習率
    warmup_steps=config.warmup_steps,  # ウォームアップステップ数
)
```

</div>
</details>

In [ ]:
training_args = TrainingArguments(
    output_dir="output",  # 出力ディレクトリ
    overwrite_output_dir=True,  # 出力ディレクトリを上書きする
    report_to="none",  # ログの報告先
    num_train_epochs=config.n_epochs,  # トレーニングエポック数
    per_device_train_batch_size=config.per_device_train_batch_size,  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps=config.gradient_accumulation_steps,  # グラデーション蓄積ステップ数
    per_device_eval_batch_size=config.per_device_eval_batch_size,  # デバイスごとの評価バッチサイズ
    logging_steps=10,  # ログの出力間隔
    eval_strategy="epoch",  # 評価戦略
    save_strategy="steps",  # 保存戦略
    save_steps=200,  # 保存ステップ数
    optim=config.optim_type,  # オプティマイザの種類
    fp16=True,  # FP16を使用するか
    learning_rate=config.lr,  # 学習率
    warmup_steps=config.warmup_steps,  # ウォームアップステップ数
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

#### LoRA config

</div>
<div class="column-right">

# 日本語訳

#### LoRA設定


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
lora_config = LoraConfig(
    r=config.lora_r,  # LoRAのランク
    lora_alpha=config.lora_alpha,  # LoRAのアルファ
    # 自己注意のみにターゲットを絞る
    target_modules=["q_proj", "k_proj", "v_proj"],  # ターゲットモジュール
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],  # 変換対象レイヤー
    lora_dropout=config.lora_dropout,  # LoRAのドロップアウト率
    bias=config.lora_bias,  # LoRAのバイアス
    task_type=TaskType.SEQ_CLS,  # タスクの種類
)
```

</div>
</details>

In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,  # LoRAのランク
    lora_alpha=config.lora_alpha,  # LoRAのアルファ
    # 自己注意のみにターゲットを絞る
    target_modules=["q_proj", "k_proj", "v_proj"],  # ターゲットモジュール
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],  # 変換対象レイヤー
    lora_dropout=config.lora_dropout,  # LoRAのドロップアウト率
    bias=config.lora_bias,  # LoRAのバイアス
    task_type=TaskType.SEQ_CLS,  # タスクの種類
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Instantiate the tokenizer & model

</div>
<div class="column-right">

# 日本語訳

### トークナイザーとモデルのインスタンス化


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"
```

</div>
<div class="column-right">

# 日本語訳

```python
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)  # トークナイザーの読み込み
tokenizer.add_eos_token = True  # <eos>を末尾に追加
tokenizer.padding_side = "right"  # パディングを右側に設定
```

</div>
</details>

In [ ]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)  # トークナイザーの読み込み
tokenizer.add_eos_token = True  # <eos>を末尾に追加
tokenizer.padding_side = "right"  # パディングを右側に設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model
```

</div>
<div class="column-right">

# 日本語訳

```python
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,  # モデルの読み込み
    num_labels=3,  # 分類ラベルの数
    torch_dtype=torch.float16,  # データ型の設定
    device_map="auto",  # デバイスマッピング
)
model.config.use_cache = False  # キャッシュを使用しない
model = prepare_model_for_kbit_training(model)  # kビットトレーニングのためのモデル準備
model = get_peft_model(model, lora_config)  # LoRAモデルの取得
model  # モデルを表示
```

</div>
</details>

In [ ]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,  # モデルの読み込み
    num_labels=3,  # 分類ラベルの数
    torch_dtype=torch.float16,  # データ型の設定
    device_map="auto",  # デバイスマッピング
)
model.config.use_cache = False  # キャッシュを使用しない
model = prepare_model_for_kbit_training(model)  # kビットトレーニングのためのモデル準備
model = get_peft_model(model, lora_config)  # LoRAモデルの取得
model  # モデルを表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model.print_trainable_parameters()
```

</div>
<div class="column-right">

# 日本語訳

```python
model.print_trainable_parameters()  # トレーニング可能なパラメータを表示
```

</div>
</details>

In [ ]:
model.print_trainable_parameters()  # トレーニング可能なパラメータを表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Instantiate the dataset

</div>
<div class="column-right">

# 日本語訳

### データセットのインスタンス化


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
ds = ds.select(torch.arange(100))  # We only use the first 100 data for demo purpose
```

</div>
<div class="column-right">

# 日本語訳

```python
ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")  # CSVからデータセットを作成
ds = ds.select(torch.arange(100))  # デモ目的で最初の100データのみを使用
```

</div>
</details>

In [ ]:
ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")  # CSVからデータセットを作成
ds = ds.select(torch.arange(100))  # デモ目的で最初の100データのみを使用

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))
```

</div>
<div class="column-right">

# 日本語訳

```python
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer  # トークナイザーの初期化
        self.max_length = max_length  # 最大長の設定
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]  # プロンプトの整形
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]  # 応答Aの整形
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]  # 応答Bの整形
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # プロンプトと応答を結合
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)  # トークナイズ
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:  # モデルAが勝った場合
                label = 0  # ラベルは0
            elif b_win:  # モデルBが勝った場合
                label = 1  # ラベルは1
            else:  # 同点の場合
                label = 2  # ラベルは2
            labels.append(label)  # ラベルをリストに追加
        return {**tokenized, "labels": labels}  # トークン化されたデータとラベルを返す
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))  # テキストを処理して返す
```

</div>
</details>

In [ ]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer  # トークナイザーの初期化
        self.max_length = max_length  # 最大長の設定
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]  # プロンプトの整形
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]  # 応答Aの整形
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]  # 応答Bの整形
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # プロンプトと応答を結合
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)  # トークナイズ
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:  # モデルAが勝った場合
                label = 0  # ラベルは0
            elif b_win:  # モデルBが勝った場合
                label = 1  # ラベルは1
            else:  # 同点の場合
                label = 2  # ラベルは2
            labels.append(label)  # ラベルをリストに追加
        return {**tokenized, "labels": labels}  # トークン化されたデータとラベルを返す
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))  # テキストを処理して返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)
```

</div>
<div class="column-right">

# 日本語訳

```python
encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンス化
ds = ds.map(encode, batched=True)  # データセットにマッピング
```

</div>
</details>

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンス化
ds = ds.map(encode, batched=True)  # データセットにマッピング

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Compute metrics

We'll compute the log-loss used in LB and accuracy as a auxiliary metric.

</div>
<div class="column-right">

# 日本語訳

### 指標の計算

LBで使用するロスと補助的な指標として精度を計算します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}
```

</div>
<div class="column-right">

# 日本語訳

```python
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # 予測値の取得
    labels = eval_preds.label_ids  # ラベルの取得
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # 予測値をソフトマックスで確率に変換
    loss = log_loss(y_true=labels, y_pred=probs)  # ロスの計算
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 精度の計算
    return {"acc": acc, "log_loss": loss}  # 精度とロスを返す
```

</div>
</details>

In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # 予測値の取得
    labels = eval_preds.label_ids  # ラベルの取得
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # 予測値をソフトマックスで確率に変換
    loss = log_loss(y_true=labels, y_pred=probs)  # ロスの計算
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 精度の計算
    return {"acc": acc, "log_loss": loss}  # 精度とロスを返す

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Split

Here, train and eval is splitted according to their `id % 5`

</div>
<div class="column-right">

# 日本語訳

### データ分割

ここでは、トレーニングと評価を`id % 5`に基づいて分割します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    ) 
    for fold_idx in range(config.n_splits)
]
```

</div>
<div class="column-right">

# 日本語訳

```python
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],  # トレーニングインデックス
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]  # 評価インデックス
    ) 
    for fold_idx in range(config.n_splits)
]
```

</div>
</details>

In [ ]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],  # トレーニングインデックス
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]  # 評価インデックス
    ) 
    for fold_idx in range(config.n_splits)
]

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args, 
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()
```

</div>
<div class="column-right">

# 日本語訳

```python
train_idx, eval_idx = folds[config.fold_idx]  # トレーニングと評価のインデックス取得

trainer = Trainer(
    args=training_args,  # トレーナー引数
    model=model,  # モデル
    tokenizer=tokenizer,  # トークナイザー
    train_dataset=ds.select(train_idx),  # トレーニングデータセット
    eval_dataset=ds.select(eval_idx),  # 評価データセット
    compute_metrics=compute_metrics,  # 指標計算関数
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # データのコラトレータ
)
trainer.train()  # トレーニングを開始
```

</div>
</details>

In [ ]:
train_idx, eval_idx = folds[config.fold_idx]  # トレーニングと評価のインデックス取得

trainer = Trainer(
    args=training_args,  # トレーナー引数
    model=model,  # モデル
    tokenizer=tokenizer,  # トークナイザー
    train_dataset=ds.select(train_idx),  # トレーニングデータセット
    eval_dataset=ds.select(eval_idx),  # 評価データセット
    compute_metrics=compute_metrics,  # 指標計算関数
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # データのコラトレータ
)
trainer.train()  # トレーニングを開始